## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-21-01-15-04 +0000,nyt,The F.C.C. chair questioned the BBC over edits...,https://www.nytimes.com/live/2025/11/20/us/tru...
1,2025-11-21-01-09-00 +0000,nyt,Trump Calls Democrats’ Military Video ‘Seditio...,https://www.nytimes.com/2025/11/20/us/politics...
2,2025-11-21-01-08-49 +0000,bbc,US and Asian stocks slide as AI jitters persist,https://www.bbc.com/news/articles/cwy1yl1202yo...
3,2025-11-21-01-08-00 +0000,wsj,Air-Traffic Controllers With Perfect Shutdown ...,https://www.wsj.com/politics/policy/air-traffi...
4,2025-11-21-01-08-00 +0000,wsj,The Transportation Department said it would pa...,https://www.wsj.com/politics/policy/air-traffi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2362/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,trump,57
174,new,24
67,ukraine,16
515,epstein,15
103,house,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
399,2025-11-20-01-41-44 +0000,wapo,U.S. pushes new Ukraine peace plan as Trump’s ...,https://www.washingtonpost.com/national-securi...,131
102,2025-11-20-22-15-38 +0000,cbc,Trump administration touts plan for new oil dr...,https://www.cbc.ca/news/world/trump-administra...,128
394,2025-11-20-02-14-00 +0000,wsj,Trump Administration Pushes New Plan for Endin...,https://www.wsj.com/world/trump-administration...,126
389,2025-11-20-02-32-00 +0000,wsj,Trump Says He Will Meet With Mamdani Friday at...,https://www.wsj.com/politics/elections/trump-s...,111
94,2025-11-20-22-26-31 +0000,bbc,Trump signs bill ordering US justice departmen...,https://www.bbc.com/news/articles/cly2y40lj5go...,110


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
399,131,2025-11-20-01-41-44 +0000,wapo,U.S. pushes new Ukraine peace plan as Trump’s ...,https://www.washingtonpost.com/national-securi...
378,68,2025-11-20-03-35-00 +0000,wsj,Congress’s approval of legislation to force a ...,https://www.wsj.com/politics/policy/epstein-fi...
209,62,2025-11-20-17-20-47 +0000,nypost,Kamala Harris and Jill Biden come face-to-face...,https://nypost.com/2025/11/20/us-news/kamala-h...
389,54,2025-11-20-02-32-00 +0000,wsj,Trump Says He Will Meet With Mamdani Friday at...,https://www.wsj.com/politics/elections/trump-s...
33,48,2025-11-20-23-59-45 +0000,nypost,Mayor Eric Adams’ administration comes out swi...,https://nypost.com/2025/11/20/us-news/mayor-er...
4,47,2025-11-21-01-08-00 +0000,wsj,The Transportation Department said it would pa...,https://www.wsj.com/politics/policy/air-traffi...
348,45,2025-11-20-08-35-04 +0000,wsj,Stock Market Today: Stock-Market Rebound Evapo...,https://www.wsj.com/livecoverage/jobs-report-b...
198,42,2025-11-20-17-55-23 +0000,nypost,Israeli strikes kill 27 in Gaza after alleged ...,https://nypost.com/2025/11/20/world-news/israe...
141,37,2025-11-20-20-38-00 +0000,nypost,Judge orders Trump to end National Guard deplo...,https://nypost.com/2025/11/20/us-news/judge-or...
361,36,2025-11-20-06-36-15 +0000,nypost,Terminally ill 25-year-old chooses to die on ‘...,https://nypost.com/2025/11/20/world-news/termi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
